<a href="https://colab.research.google.com/github/hbrown227/BiologistScripts/blob/master/Chlamydia_Clustering_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install dependences and call googlesheet

In [ ]:
from gc import get_freeze_count
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yellowbrick.cluster
import sklearn

from sklearn.cluster import KMeans, SpectralClustering
from sklearn.metrics import silhouette_samples, silhouette_score
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
from yellowbrick.cluster import KElbowVisualizer

from matplotlib import font_manager
from google.auth.credentials import Credentials
%matplotlib inline
plt.style.use('seaborn-whitegrid')
font_manager.FontManager({'font.sans-serif':'Arial'})


from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

Open worksheet and covert to dataframe then array

In [ ]:

worksheet = gc.open('2230hpi').sheet1

df= pd.DataFrame(worksheet.get_all_records())

print(df.head)



DBscan

In [ ]:
from sklearn.cluster import DBSCAN

X = np.array([[1, 2], [2, 2], [2, 3],
              [8, 7], [8, 8], [25, 80]])
clustering = DBSCAN(eps=3, min_samples=2).fit(X)
clustering.labels_

clustering

fit(X, y=None, sample_weight=None)

Elbow Method for KMeans

In [ ]:


# define min max scaler
## need to normalize data in order to not have outsized effect by one of the parameters
scaler = MinMaxScaler()

print(scaler.fit(df[['euo','hctB']]))

print(scaler.data_max_)

print(scaler.transform(df[['euo','hctB']]))


model= KMeans()
visualizer = KElbowVisualizer (model, k=(1,10))
visualizer.fit(df)
visualizer.show()
visualizer.elbow_value_


Bisect

In [ ]:
clustering = SpectralClustering(n_clusters=2,
        assign_labels='discretize',
        random_state=0).fit(df)
clustering.labels_

clustering



In [ ]:

k_means_optimum = KMeans(n_clusters = 3, init = 'k-means++',  random_state=42)
y = k_means_optimum.fit_predict(df[['euo','hctB','Volume']])
print(y)
pd.DataFrame(y).to_csv("classified.csv")
from google.colab import files
files.download("classified.csv")


Visualize

In [ ]:

HctA = worksheet.col_values(2)
HctA.pop(0)
#[float(i) for i in HctB]
HctA = np.array(HctA, dtype=np.float32)


#gene2
Euo = worksheet.col_values(1)
Euo.pop(0)
#[float(i) for i in Euo]
Euo = np.array(Euo,dtype=np.float32)


#volume
size = worksheet.col_values(3)
size.pop(0)

#[float(i) for i in size]
size = np.array(size, dtype=np.float32)
size = 40 * size

#make graph 
##x=Euo, y=HctB, s=size, c=y (kmeans cluster), cmap=what colors to use, alpha=transperancy
plt.scatter(Euo, HctA, s=size, c=y, cmap='viridis', alpha=0.5,)
plt.axis(xmin=0,xmax=2500,ymin=0, ymax=10000)
plt.xlabel('Euo',fontweight ="bold")
plt.ylabel('HctA',fontweight ="bold")
plt.title('30hpi',fontweight ="bold")

from google.colab import files
plt.savefig('30hpi.pdf')
plt.show()
files.download('30hpi.pdf') 


In [ ]:
plt.scatter(Euo, HctB, s=size, c=y, cmap='viridis', alpha=0.5,)

plt.xscale("log")
plt.yscale("log")

plt.xlabel('Euo',fontweight ="bold")
plt.ylabel('HctB',fontweight ="bold")
plt.title('30hpi',fontweight ="bold")

plt.savefig('30hpi_log.pdf')
plt.show()
files.download('30hpi_log.pdf') 
